In [1]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import scipy.stats
import numpy as np
import os

In [2]:
metadata = pd.read_excel('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/Tables/1_Metdata_v2.1.xlsx')
requant = pd.read_excel('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/Tables/3_Requantification_v1.3.xlsx', index_col=0).reset_index(drop=True)

In [3]:
# Individual deduplication : keep the sample with the highest RPM from same individual
requant_indvrmdup = requant[['sn', 'vsname', 'Inferred_Individual', 'RPM']].sort_values('RPM', ascending=False).drop_duplicates(subset=['vsname', 'Inferred_Individual'], keep='first')
requant_indvrmdup

,sn,vsname,Inferred_Individual,RPM
288,R2011008079,AstV-16A,BAT256,533906.839082
731,R2101013076,ParV-1A,BRA2271,235245.473705
730,R2101013052,ParV-1A,BRA2246,62463.818710
729,R2101012929,ParV-1A,BRA2019,45300.307156
506,R2007002175,CalV-2A,BAT784,42015.537360
...,...,...,...,...
665,R2011005935,ParV-7A,UV28,0.568081
797,R2011005940,AstV-2A,UV33,0.561210
664,R2011005944,ParV-7A,UV37,0.552419
22,R2007002177,FlaV-8A,19BR182,0.527513


In [4]:
def vcf2dataframe(vcfpath, parse_info=False, parse_annot=False):
    '''
    读取一个vcf转换成dataframe 列与vcfheader的列一致
    '''
    if not parse_info and parse_annot:
        parse_info = True
        print('[WARNING] When parse_annot=True, parse_info will be forced set to True!')
    for line in open(vcfpath).readlines():
        if line.startswith('#CHROM'):
            vcfdfheader = line.lstrip('#').split()
            break
    vcfdf = pd.read_table(vcfpath, comment='#', header=None, names=vcfdfheader)
    if len(vcfdf)==0:
        return vcfdf
    if parse_info:
        vcfinfo = pd.DataFrame(list(vcfdf['INFO'].apply(lambda x: { pair.split('=')[0]:pair.split('=')[1] for pair in x.split(';') })))
        vcfdf = pd.concat([vcfdf, vcfinfo], axis=1)
        if parse_annot: # parse annotation from snpEff
            ann_cols = 'Allele | Annotation | Annotation_Impact | Gene_Name | Gene_ID | Feature_Type | Feature_ID | Transcript_BioType | Rank | HGVS.c | HGVS.p | cDNA.pos / cDNA.length | CDS.pos / CDS.length | AA.pos / AA.length | Distance | ERRORS / WARNINGS / INFO'.split(' | ')
            vcfann = pd.DataFrame(list(vcfdf['ANN'].apply(lambda x: x.split('|', 15))), columns = ann_cols)
            vcfdf = pd.concat([vcfdf.drop('ANN', axis=1), vcfann], axis=1)
    return vcfdf

Process iSNV


In [5]:
result = []
adpp_result = []

for filepath in tqdm(glob('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/9.2.lofreq_iSNV_annotated/*/*.ALTiSNV.vcf')):
    vsname = filepath.split('/')[-2]
    sn = filepath.split('/')[-1].split('.')[0]
    vcfdf_ann = vcf2dataframe(filepath, parse_info=True, parse_annot=True)
    posnum = len(vcfdf_ann)
    if not posnum: # Skip empty file
        continue
    vcfdf_ann = vcfdf_ann[vcfdf_ann['iSNV']=='ALT']
    vcfdf_ann['ADPP'] = vcfdf_ann['DP4'].apply(lambda x: int(x.split(',')[2]) + int(x.split(',')[3])) / vcfdf_ann['DP'].astype(int)   # ADPP: Allele DP Percentage
    altposnum = len(vcfdf_ann)
    synonymous_vcfdf = vcfdf_ann[vcfdf_ann['Annotation']=='synonymous_variant']
    synonymous_meandp = synonymous_vcfdf['ADPP'].astype(float).mean()
    missense_vcfdf = vcfdf_ann[vcfdf_ann['Annotation']=='missense_variant']
    missense_meandp = missense_vcfdf['ADPP'].astype(float).mean()
    vcfdf_ann['CHROM'] = vsname
    vcfdf_ann['SN'] = sn
    
    adpp_result.append(vcfdf_ann[['SN','CHROM', 'POS', 'REF', 'ALT', 'ADPP', 'Annotation']])
    result.append([vsname, sn, posnum, altposnum, len(synonymous_vcfdf), synonymous_meandp, len(missense_vcfdf), missense_meandp])


100%|██████████| 222/222 [00:04<00:00, 54.48it/s]


In [6]:
allsite_isnvdp = pd.concat(adpp_result) # table with every single iSNV POS info 
allsite_isnvdp

,SN,CHROM,POS,REF,ALT,ADPP,Annotation
0,R2011005870,PicoV-24A,4328,C,T,0.064286,missense_variant
1,R2011005870,PicoV-24A,5889,T,C,0.080645,synonymous_variant
0,R2011005891,PicoV-24A,660,A,C,0.101449,synonymous_variant
1,R2011005891,PicoV-24A,4587,T,C,0.304878,synonymous_variant
2,R2011005891,PicoV-24A,5448,T,C,0.122642,synonymous_variant
...,...,...,...,...,...,...,...
10,R2101012916,CV-6A,1399,A,G,0.427350,missense_variant
11,R2101012916,CV-6A,1405,A,T,0.356436,missense_variant
19,R2101012916,CV-6A,1814,T,C,0.340659,synonymous_variant
0,R2101012909,CV-6A,1387,C,A,0.160987,missense_variant


In [7]:
allsite_isnvdp

,SN,CHROM,POS,REF,ALT,ADPP,Annotation
0,R2011005870,PicoV-24A,4328,C,T,0.064286,missense_variant
1,R2011005870,PicoV-24A,5889,T,C,0.080645,synonymous_variant
0,R2011005891,PicoV-24A,660,A,C,0.101449,synonymous_variant
1,R2011005891,PicoV-24A,4587,T,C,0.304878,synonymous_variant
2,R2011005891,PicoV-24A,5448,T,C,0.122642,synonymous_variant
...,...,...,...,...,...,...,...
10,R2101012916,CV-6A,1399,A,G,0.427350,missense_variant
11,R2101012916,CV-6A,1405,A,T,0.356436,missense_variant
19,R2101012916,CV-6A,1814,T,C,0.340659,synonymous_variant
0,R2101012909,CV-6A,1387,C,A,0.160987,missense_variant


In [8]:
allsite_isnvdp = allsite_isnvdp.merge(requant_indvrmdup[['sn', 'vsname']], left_on=['SN', 'CHROM'], right_on=['sn', 'vsname'], how='inner')
allsite_isnvdp.to_excel('iSNV_full_perPOS_annotated.xlsx', index=False)

In [9]:
# Unfiltered iSNV num stat
allsite_isnvdp[['CHROM', 'POS']].drop_duplicates().groupby('CHROM').count().sort_values(by='POS')

,POS
CHROM,
HerpV-4A,1
CalV-21A,1
PicoV-17B,1
CalV-5A,1
ParV-11A,1
...,...
ParV-1A,188
PV-12A,240
CalV-15A,299


In [10]:
allsite_isnvdp_missense = allsite_isnvdp[allsite_isnvdp['Annotation']=='missense_variant']
allsite_isnvdp_missense

,SN,CHROM,POS,REF,ALT,ADPP,Annotation,sn,vsname
0,R2011005870,PicoV-24A,4328,C,T,0.064286,missense_variant,R2011005870,PicoV-24A
5,R2011005891,PicoV-24A,5457,G,T,0.264151,missense_variant,R2011005891,PicoV-24A
6,R2011005891,PicoV-24A,6197,G,T,0.125000,missense_variant,R2011005891,PicoV-24A
28,R2011008083,CV-12A,1171,T,A,0.219643,missense_variant,R2011008083,CV-12A
29,R2011008083,CV-12A,1186,A,T,0.244737,missense_variant,R2011008083,CV-12A
...,...,...,...,...,...,...,...,...,...
18915,R2101012916,CV-6A,1208,C,A,0.164087,missense_variant,R2101012916,CV-6A
18916,R2101012916,CV-6A,1399,A,G,0.427350,missense_variant,R2101012916,CV-6A
18917,R2101012916,CV-6A,1405,A,T,0.356436,missense_variant,R2101012916,CV-6A
18919,R2101012909,CV-6A,1387,C,A,0.160987,missense_variant,R2101012909,CV-6A


In [11]:
allsite_isnvdp_synonymous = allsite_isnvdp[allsite_isnvdp['Annotation']=='synonymous_variant']
allsite_isnvdp_synonymous

,SN,CHROM,POS,REF,ALT,ADPP,Annotation,sn,vsname
1,R2011005870,PicoV-24A,5889,T,C,0.080645,synonymous_variant,R2011005870,PicoV-24A
2,R2011005891,PicoV-24A,660,A,C,0.101449,synonymous_variant,R2011005891,PicoV-24A
3,R2011005891,PicoV-24A,4587,T,C,0.304878,synonymous_variant,R2011005891,PicoV-24A
4,R2011005891,PicoV-24A,5448,T,C,0.122642,synonymous_variant,R2011005891,PicoV-24A
7,R2101013357,CalV-11A,3405,C,T,0.145695,synonymous_variant,R2101013357,CalV-11A
...,...,...,...,...,...,...,...,...,...
18906,R2101013376,CoV-4B,1368,T,C,0.482143,synonymous_variant,R2101013376,CoV-4B
18907,R2101013376,CoV-4B,24498,C,T,0.097087,synonymous_variant,R2101013376,CoV-4B
18912,R2101012916,CV-6A,208,G,T,0.396947,synonymous_variant,R2101012916,CV-6A
18913,R2101012916,CV-6A,379,A,G,0.180645,synonymous_variant,R2101012916,CV-6A


In [13]:
isnv_stat = pd.DataFrame(result, columns=['vsname', 'sn', 'num_iSNV', 'num_ALTiSNV', 'synonymous_varnum', 'synonymous_mean', 'nonsynonymous_varnum', 'nonsynonymous_mean']) # 
isnv_stat

,vsname,sn,num_iSNV,num_ALTiSNV,synonymous_varnum,synonymous_mean,nonsynonymous_varnum,nonsynonymous_mean
0,PicoV-24A,R2011005870,2,2,1,0.080645,1,0.064286
1,PicoV-24A,R2011005891,5,5,3,0.176323,2,0.194575
2,CalV-11A,R2101013357,3,3,3,0.105905,0,NaN
3,CalV-11A,R2101013362,7,7,7,0.187506,0,NaN
4,CalV-11A,R2101013359,6,5,5,0.398594,0,NaN
...,...,...,...,...,...,...,...,...
217,ParaV-7A,R2012012418,1,1,0,NaN,1,0.115385
218,ParaV-7A,R2012012434,2,2,0,NaN,2,0.072944
219,AstV-14B,R2012012518,1,1,0,NaN,1,0.307692
220,CV-6A,R2101012916,24,7,3,0.306084,4,0.316236


In [14]:
isnv_stat['num_iSNV'].sum()

32631

In [15]:
isnv_stat.to_excel('iSNV_variantDPstat.xlsx', index=None)

In [15]:
isnv_stat.dropna()

,vsname,num_varsite,samplenum,synonymous_varnum,synonymous_mean,nonsynonymous_varnum,nonsynonymous_mean
0,PicoV-24A,R2011005870,2,1,0.080645,1,0.064286
1,PicoV-24A,R2011005891,5,3,0.176323,2,0.194575
8,PicoV-4A,R2007002137,20,16,0.112079,1,0.129496
9,PicoV-4A,R2011008081,6,4,0.245958,1,0.170846
11,HerpV-2A,R2011008212,89,39,0.291837,15,0.317799
...,...,...,...,...,...,...,...
212,ReoV-2B,R2012012471,2,1,0.259259,1,0.070175
213,PolV-5A,R2011005797,147,83,0.245467,37,0.229185
214,CoV-4B,R2101013364,15,9,0.305226,4,0.144638
215,CoV-4B,R2101013369,3,1,0.060606,2,0.150083


In [134]:
scipy.stats.wilcoxon(list(isnv_stat.dropna()['synonymous_mean']), list(isnv_stat.dropna()['nonsynonymous_mean']))

WilcoxonResult(statistic=3074.0, pvalue=0.012190893096740658)

In [148]:
scipy.stats.mannwhitneyu(list(allsite_isnvdp_synonymous['ADPP']), list(allsite_isnvdp_missense['ADPP']))

MannwhitneyuResult(statistic=27783595.5, pvalue=0.5248744974058455)

Remove Coinfection

In [16]:
vsnamelist = list(map(lambda x: x.split('.')[0], os.listdir('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/refiltering_test/filtered_annotated')))

In [17]:
def parsing_sitepos(vcfpath):
    varpos = []
    for line in open(vcfpath).readlines():
        if line.startswith('#'):
            continue
        else:
            chrom, pos, id, ref, alt, *_ = line.strip().split()
            varpos.append(int(pos))
    return varpos


In [18]:
def compare_sites(isnvdict, fbvcfdict, coinfect_threshold=3):
    '''
    比较每个样本的isnv和csnv
    '''
    output_list = []
    for isnvsn, isnvsites in isnvdict.items():
        for csnvsn, csnvsites in fbvcfdict.items():
            isec_sites = set(isnvsites) & set(csnvsites)
            if len(isec_sites) >= coinfect_threshold:
                output_list.append([isnvsn, csnvsn, len(isec_sites)])
    outputdf = pd.DataFrame(output_list, columns=['iSNV_sn', 'cSNV_sn', 'isec_sites'])
    return outputdf

In [22]:
coinfect_resultlist = []
fbvcf_stat = []

for vsname in vsnamelist:
    fbvcfpath = '/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/refiltering_test/filtered_annotated/{vsname}.snpeff.vcf'.format(vsname=vsname)
    if not os.path.exists('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/9.2.lofreq_iSNV_annotated/{vsname}'.format(vsname=vsname)):
        continue
    fbvcfdf = vcf2dataframe(fbvcfpath)
    requantsn = set(requant_indvrmdup.loc[requant_indvrmdup['vsname']==vsname, 'sn'])
    fbsnlist = set(fbvcfdf.columns[9:])
    kept_fbsnlist = requantsn & fbsnlist
    fbvcf_stat.append([vsname, len(kept_fbsnlist), len(fbvcfdf)])
    # convert cSNV for every single individual (1:...) into dictionary, format as {sn1:[POS1,POS2,POS3....],sn2:[...]}
    fbvcfvardict = { sn:list(fbvcfdf.loc[fbvcfdf[sn].apply(lambda x: x.startswith('1:')), 'POS']) for sn in kept_fbsnlist }  
    lofreqvcflist = glob('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/9.2.lofreq_iSNV_annotated/{vsname}/*.vcf'.format(vsname=vsname))
    # convert iSNV called by lofreq into dictionary, using the same format as cSNV
    isnvdict = { filepath.split('/')[-1].split('.')[0]:list(vcf2dataframe(filepath)['POS']) for filepath in lofreqvcflist }
    compare_df = compare_sites(isnvdict, fbvcfvardict, coinfect_threshold=3) # compare variation sites between iSNV and cSNV，at least 3 variant at same position will considered as coinfection event
    compare_df['vsname'] = vsname
    coinfect_resultlist.append(compare_df)
    
coinfect_resultdf = pd.concat(coinfect_resultlist)

In [23]:
fbvcf_statdf = pd.DataFrame(fbvcf_stat, columns=['vsname', 'num_sample', 'num_sites'])
fbvcf_statdf_filtered = fbvcf_statdf[(fbvcf_statdf['num_sample']>4) & (fbvcf_statdf['num_sites']>20)].copy()
fbvcf_statdf_filtered['vsname'].to_csv('ld_kept_vsname.list', sep='\t')

In [24]:
coinfect_resultdf_dropself = coinfect_resultdf[coinfect_resultdf['iSNV_sn']!=coinfect_resultdf['cSNV_sn']].copy()   # 去除同一个样本发现的iSNV和cSNV共感染组合
coinfect_resultdf_dropself

,iSNV_sn,cSNV_sn,isec_sites,vsname
0,R2011005922,R2101013343,4,PicoV-10A
1,R2011005922,R2101013353,5,PicoV-10A
2,R2011005922,R2011005932,3,PicoV-10A
3,R2011005922,R2011005931,3,PicoV-10A
0,R2011005939,R2101013342,10,AstV-2A
...,...,...,...,...
1,R2011005924,R2011005936,4,CV-8B
0,R2011008097,R2011008078,3,AdV-2A
2,R2011008097,R2011008080,8,AdV-2A
0,R2012012472,R2012012462,4,ParV-2A


In [14]:
coinfect_list = list(coinfect_resultdf_dropself['vsname'] + ' ' + coinfect_resultdf_dropself['iSNV_sn'])

In [15]:
coinfect_list

['PicoV-10A R2011005922',
 'PicoV-10A R2011005922',
 'PicoV-10A R2011005922',
 'PicoV-10A R2011005922',
 'AstV-2A R2011005939',
 'AstV-2A R2011005939',
 'ParV-26A R2101013380',
 'ParV-26A R2101013380',
 'ParV-26A R2101013381',
 'ParV-26A R2101013381',
 'ParV-26A R2101013377',
 'ParV-26A R2101013377',
 'CV-14A R2101013369',
 'CV-14A R2101013369',
 'CV-14A R2101013369',
 'CV-14A R2101013376',
 'CV-14A R2101013376',
 'CV-14A R2101013376',
 'CV-14A R2101013376',
 'CV-14A R2101013365',
 'CV-14A R2101013365',
 'AstV-15A R2012012506',
 'AstV-15A R2012012506',
 'HanV-1A R2101013353',
 'HanV-1A R2101013353',
 'HanV-1A R2101013343',
 'HanV-1A R2101013343',
 'CV-3A R2101013365',
 'CV-3A R2101013365',
 'CV-3A R2101013365',
 'CalV-13A R2101013353',
 'CalV-13A R2101013353',
 'CalV-13A R2101013353',
 'CalV-13A R2101013353',
 'CalV-13A R2101013353',
 'CalV-13A R2103020334',
 'ParV-21A R2101013401',
 'ParV-21A R2101013401',
 'ParV-21A R2101013401',
 'PicoV-17A R2012012566',
 'PicoV-17A R2012012566',
 '

统计iSNV数量

In [16]:
kept_pair = list(requant_indvrmdup['vsname'] + ' ' + requant_indvrmdup['sn'])

In [29]:
result = []
adpp_result = []
coinfection_result = []

for filepath in tqdm(glob('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/9.2.lofreq_iSNV_annotated/*/*.ALTiSNV.vcf')):
    vsname = filepath.split('/')[-2]
    sn = filepath.split('/')[-1].split('.')[0]
    if not vsname + ' ' + sn in kept_pair:
        print(vsname + ' ' + sn + ' removed(indv-dedup)!') 
        continue 
    if vsname + ' ' + sn in coinfect_list:   # 如果该vs-sn pair出现在共感染列表或未出现在保留的pair中，则去除
        print(vsname + ' ' + sn + ' removed!')
        coinfection_result.append([vsname, sn, True])
        continue
    else:
        coinfection_result.append([vsname, sn, False])
    
    vcfdf_ann = vcf2dataframe(filepath, parse_info=True, parse_annot=True)
    posnum = len(vcfdf_ann)
    if not posnum:
        continue
    vcfdf_ann = vcfdf_ann[vcfdf_ann['iSNV']=='ALT']
    vcfdf_ann['ADPP'] = vcfdf_ann['DP4'].apply(lambda x: int(x.split(',')[2]) + int(x.split(',')[3])) / vcfdf_ann['DP'].astype(int)
    
    altposnum = len(vcfdf_ann)
    
    synonymous_vcfdf = vcfdf_ann[vcfdf_ann['Annotation']=='synonymous_variant']
    synonymous_meandp = synonymous_vcfdf['ADPP'].astype(float).mean()
    missense_vcfdf = vcfdf_ann[vcfdf_ann['Annotation']=='missense_variant']
    missense_meandp = missense_vcfdf['ADPP'].astype(float).mean()
    vcfdf_ann['CHROM'] = vsname
    vcfdf_ann['SN'] = sn
    adpp_result.append(vcfdf_ann[['SN','CHROM', 'POS', 'REF', 'ALT', 'ADPP', 'Annotation']])
    
    result.append([vsname, sn, posnum, altposnum, len(synonymous_vcfdf), synonymous_meandp, len(missense_vcfdf), missense_meandp])

 11%|█▏        | 25/222 [00:00<00:01, 128.74it/s]

HerpV-2A R2011008212 removed!
HerpV-2A R2101013266 removed!
HerpV-2A R2011008225 removed!
HerpV-2A R2011008211 removed!
HerpV-2A R2011008219 removed!
HerpV-2A R2011008210 removed!
HerpV-2A R2011008220 removed!
HerpV-2A R2011008223 removed!
HerpV-2A R2011008221 removed!
AdV-2A R2011008097 removed!


 27%|██▋       | 60/222 [00:00<00:01, 86.79it/s] 

CV-14A R2101013369 removed!
CV-14A R2101013376 removed!
CV-14A R2101013365 removed!


 36%|███▌      | 79/222 [00:00<00:01, 87.94it/s]

...

 84%|████████▍ | 187/222 [00:01<00:00, 90.70it/s] 

CoV-8A R2012012521 removed!
ParV-26A R2101013380 removed!
ParV-26A R2101013381 removed!
ParV-26A R2101013377 removed!


 94%|█████████▎| 208/222 [00:02<00:00, 84.13it/s]

ParV-1A R2101013032 removed(indv-dedup)!


100%|██████████| 222/222 [00:02<00:00, 90.38it/s]


In [36]:
coinfdf = pd.DataFrame(coinfection_result, columns=['vsname', 'sn', 'coinfection']).merge(requantdf[['vsname', 'sn', 'meandepth', 'coverage']], on=['vsname', 'sn'], how='left')
coinfdf = coinfdf[(coinfdf['meandepth']>50)]
coinfdf.to_csv('CoInfection_stat.csv', index=False)

In [35]:
set(coinfdf[coinfdf['coinfection']]['vsname'])

{'AdV-2A',
 'AstV-15A',
 'AstV-2A',
 'AstV-9A',
 'CV-14A',
 'CV-3A',
 'CV-8B',
 'CalV-13A',
 'CalV-15A',
 'CoV-3A',
 'HanV-1A',
 'HerpV-2A',
 'ParV-26A',
 'ParV-2A',
 'ParV-5A',
 'PicoV-17A',
 'PicoV-6A',
 'ReoV-3A'}

In [22]:
isnv_stat = pd.DataFrame(result, columns=['vsname', 'sn', 'num_iSNV', 'num_ALTiSNV', 'synonymous_ALTiSNV', 'synonymous_meanDP', 'nonsynonymous_ALTiSNV', 'nonsynonymous_meanDP'])
isnv_stat

,vsname,sn,num_iSNV,num_ALTiSNV,synonymous_ALTiSNV,synonymous_meanDP,nonsynonymous_ALTiSNV,nonsynonymous_meanDP
0,PicoV-24A,R2011005870,2,2,1,0.080645,1,0.064286
1,PicoV-24A,R2011005891,5,5,3,0.176323,2,0.194575
2,CalV-11A,R2101013357,3,3,3,0.105905,0,NaN
3,CalV-11A,R2101013362,7,7,7,0.187506,0,NaN
4,CalV-11A,R2101013359,6,5,5,0.398594,0,NaN
...,...,...,...,...,...,...,...,...
171,ParaV-7A,R2012012418,1,1,0,NaN,1,0.115385
172,ParaV-7A,R2012012434,2,2,0,NaN,2,0.072944
173,AstV-14B,R2012012518,1,1,0,NaN,1,0.307692
174,CV-6A,R2101012916,24,7,3,0.306084,4,0.316236


In [23]:
requantdf = pd.read_excel('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/VirusQuantification/requant_outjoin_v2.2.xlsx')
requantdf

,Unnamed: 0,sn,rname,startpos,endpos,numreads,covbases,coverage,meandepth,meanbaseq,...,80ani_clus,95ani_clus,Sample_ID,numreads_m,cleandata_gb,RPM,Inferred_Individual,Final_species,Sample_Type,Host_Category
0,0,R2007002118,19BR163KID-k99_18062,1,19279,152,7889,40.9202,0.934333,35.8,...,ParaV-1,ParaV-1A,19BR163KID,176.314638,23.352879,0.862095,19BR163,Otomys angoniensis,Kidney_tissue,Rat
1,1,R2007002122,FlaV-1B-Scaffold_pilon,1,9054,11307,9054,100.0000,122.025000,34.1,...,FlaV-1,FlaV-1B,19BR184KID,58.882182,5.834255,192.027531,19BR184,Lophuromys flavopunctatus,Kidney_tissue,Rat
2,2,R2007002122,19BR184KID-k119_9215,1,18012,146760,18012,100.0000,796.678000,34.1,...,ParaV-3,ParaV-3A,19BR184KID,58.882182,5.834255,2492.434808,19BR184,Lophuromys flavopunctatus,Kidney_tissue,Rat
3,3,R2007002122,19BR181KID-k99_6171,1,17571,297,11149,63.4511,1.554720,32.7,...,ParaV-5,ParaV-5A,19BR184KID,58.882182,5.834255,5.043971,19BR184,Lophuromys flavopunctatus,Kidney_tissue,Rat
4,4,R2007002128,19BR181KID-k99_6171,1,17571,262,11392,64.8341,1.414090,31.0,...,ParaV-5,ParaV-5A,2260LU,61.835300,6.124275,4.237062,2260LU,Mus bufo,Lung_tissue,Rat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,867,R2103020503,BFS094-k141_118387,1,2403,568,2403,100.0000,34.259300,35.5,...,ReoV-6,ReoV-6A,BFS094,25.904896,3.865663,21.926357,BFS094,Triaenops afer,Single_fecal_swab,Bat
868,868,R2103020506,BFS126-k141_247049,1,28849,352123,28849,100.0000,1794.840000,34.7,...,CoV-4,CoV-4D,BFS126,36.265582,5.417530,9709.564292,BFS126,Miniopterus sp. clade 5 TD-2020,Single_fecal_swab,Bat
869,869,R2103020507,BFS126-k141_247049,1,28849,120187,28768,99.7192,607.407000,35.2,...,CoV-4,CoV-4D,BFS127,7.787306,1.161016,15433.707113,BFS127,Miniopterus sp. clade 5 TD-2020,Single_fecal_swab,Bat
870,870,R2103020508,BFS128-k141_50115,1,2050,564,2050,100.0000,40.319000,34.5,...,PV-13,PV-13A,BFS128,13.945758,2.072339,40.442405,BFS128,Miniopterus sp. clade 5 TD-2020,Single_fecal_swab,Bat


In [131]:
requantdf[(requantdf['meandepth']>50) & (requantdf['vsname'] +' ' + requantdf['sn']).isin(coinfect_list)]['vfamily'].value_counts()

vfamily
Circoviridae      6
Parvoviridae      6
Herpesviridae     6
Astroviridae      3
Picornaviridae    3
Caliciviridae     3
Coronaviridae     2
Hantaviridae      2
Reoviridae        1
Adenoviridae      1
Name: count, dtype: int64

In [125]:
isnv_stat_mergedp = isnv_stat.merge(requantdf[['vsname', 'sn', 'meandepth', 'coverage']], on=['vsname', 'sn'], how='left')

In [126]:
isnv_stat_filtdp = isnv_stat_mergedp[isnv_stat_mergedp['meandepth']>=50]
isnv_stat_filtdp

,vsname,sn,num_iSNV,num_ALTiSNV,synonymous_ALTiSNV,synonymous_meanDP,nonsynonymous_ALTiSNV,nonsynonymous_meanDP,meandepth,coverage
2,CalV-11A,R2101013357,3,3,3,0.105905,0,NaN,322.8590,100.0000
3,CalV-11A,R2101013362,7,7,7,0.187506,0,NaN,628.6070,100.0000
5,CV-12A,R2011008079,5,5,1,0.076060,0,NaN,813.1420,100.0000
6,CV-12A,R2011008085,3,1,0,NaN,0,NaN,52.4100,74.5821
7,CV-12A,R2011008083,16,8,0,NaN,2,0.232190,443.6190,95.9685
...,...,...,...,...,...,...,...,...,...,...
207,ParV-1A,R2101013077,86,50,1,0.100070,0,NaN,1829.6200,86.7643
208,ReoV-2B,R2012012467,66,45,29,0.305499,1,0.076923,74.3356,98.1026
209,ReoV-2B,R2012012465,5,5,3,0.064791,2,0.070476,282.4040,100.0000
218,CV-6A,R2101012916,24,7,3,0.306084,4,0.316236,171.2540,100.0000


In [102]:
isnv_stat_filtdp.merge(allsite_isnvdp, left_on=['vsname','sn'], right_on=['CHROM','SN'], how='left').to_excel('iSNV_DPperPOS_filtered.xlsx', index=False)

In [71]:
isnv_stat_filtdp.to_excel('iSNV_DPstat_filtCoInf_filtDP_indv.xlsx', index=False)

In [70]:
isnv_stat_filtdp

,vsname,sn,num_iSNV,num_ALTiSNV,synonymous_ALTiSNV,synonymous_meanDP,nonsynonymous_ALTiSNV,nonsynonymous_meanDP,meandepth,coverage
2,CalV-11A,R2101013357,3,3,3,0.105905,0,NaN,322.8590,100.0000
3,CalV-11A,R2101013362,7,7,7,0.187506,0,NaN,628.6070,100.0000
5,CV-12A,R2011008079,5,5,1,0.076060,0,NaN,813.1420,100.0000
6,CV-12A,R2011008085,3,1,0,NaN,0,NaN,52.4100,74.5821
7,CV-12A,R2011008083,16,8,0,NaN,2,0.232190,443.6190,95.9685
...,...,...,...,...,...,...,...,...,...,...
163,ParV-1A,R2101013077,86,50,1,0.100070,0,NaN,1829.6200,86.7643
164,ReoV-2B,R2012012467,66,45,29,0.305499,1,0.076923,74.3356,98.1026
165,ReoV-2B,R2012012465,5,5,3,0.064791,2,0.070476,282.4040,100.0000
174,CV-6A,R2101012916,24,7,3,0.306084,4,0.316236,171.2540,100.0000


In [53]:
statresult = []
vardirection_stat = []
for vsname, dfvsname in isnv_stat_filtdp.groupby('vsname'):
    vs_isnvsites = set()
    for sn in list(dfvsname['sn']):
        isnv_vcfdf = vcf2dataframe('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/9.2.lofreq_iSNV_annotated/{vsname}/{sn}.ALTiSNV.vcf'.format(vsname=vsname, sn=sn))
        isnv_refseqid = isnv_vcfdf.loc[0, 'CHROM']
        var_direction = { 'vsname':vsname, 'refid':isnv_refseqid, 'sn':sn, 'num_iSNV':len(isnv_vcfdf), 'A->G':0, 'G->A':0, 'T->C':0, 'C->T':0 }
        isnv_vcfdf['vardirection'] = isnv_vcfdf['REF'] + '->' + isnv_vcfdf['ALT']
        for var_record in list(isnv_vcfdf['vardirection']):
            if var_record in var_direction.keys():
                var_direction[var_record] += 1
        isnvsites = isnv_vcfdf['POS']
        vs_isnvsites = vs_isnvsites | set(isnvsites)
        vardirection_stat.append(var_direction)
    statresult.append([vsname, len(dfvsname['sn']), len(vs_isnvsites)])

In [54]:
vardirection_df = pd.DataFrame(vardirection_stat)
vardirection_df

,vsname,refid,sn,num_iSNV,A->G,G->A,T->C,C->T
0,AdV-2A,MIX-AdV-2A,R2011008078,15,6,1,0,1
1,AdV-2A,MIX-AdV-2A,R2011008080,3,0,0,0,0
2,AstV-15A,BAT1537-k141_146819,R2012012510,8,0,1,3,3
3,AstV-15A,BAT1537-k141_146819,R2012012518,5,0,1,2,0
4,AstV-9A,BAT2999-k141_16468,R2101013365,4,2,1,0,1
...,...,...,...,...,...,...,...,...
95,PicoV-8A,UGR5-k141_333212,R2103020271,5,0,0,1,2
96,PoxV-1A,OPTM-PoxV-1A,R2103020336,2,0,0,0,0
97,ReoV-2B,BAT1219-k141_71479,R2012012467,66,22,12,12,15
98,ReoV-2B,BAT1219-k141_71479,R2012012465,5,1,0,2,1


In [18]:
vardirection_df['A<->G'] = vardirection_df['A->G'] + vardirection_df['G->A'] 
vardirection_df['C<->T'] = vardirection_df['C->T'] + vardirection_df['T->C']
vardirection_df['Transitions'] = vardirection_df['A<->G'] + vardirection_df['C<->T']
vardirection_df['Transversions'] = vardirection_df['num_iSNV'] - vardirection_df['Transitions']


In [58]:
basenum = pd.read_table('refseq_baseperc.tsv', header=None, names=['refid', 'length', r'%A', r'%T', r'%C', r'%G', r'%GC', r'%AT'])
vardirection_mergebaseperc = vardirection_df.merge(basenum, on='refid')
for vartype in list(var_direction.keys())[-4:]:
    vardirection_mergebaseperc['normalized_{vt}'.format(vt=vartype)] = vardirection_mergebaseperc[vartype] / vardirection_mergebaseperc[r'%'+vartype[0]]

In [66]:
isnv_stat_all = isnv_stat_filtdp.drop('num_iSNV', axis=1).merge(vardirection_mergebaseperc, on=['vsname', 'sn'], how='left')
isnv_stat_all

,vsname,sn,num_ALTiSNV,synonymous_ALTiSNV,synonymous_meanDP,nonsynonymous_ALTiSNV,nonsynonymous_meanDP,meandepth,coverage,refid,...,%A,%T,%C,%G,%GC,%AT,normalized_A->G,normalized_G->A,normalized_T->C,normalized_C->T
0,CalV-11A,R2101013357,3,3,0.105905,0,NaN,322.8590,100.0000,BAT2985-k141_116061,...,24.57,27.35,24.59,23.49,48.09,51.91,0.000000,0.000000,0.000000,0.122001
1,CalV-11A,R2101013362,7,7,0.187506,0,NaN,628.6070,100.0000,BAT2985-k141_116061,...,24.57,27.35,24.59,23.49,48.09,51.91,0.081400,0.042571,0.073126,0.081334
2,CV-12A,R2011008079,5,1,0.076060,0,NaN,813.1420,100.0000,BAT256-k141_32339,...,26.35,30.38,22.57,20.70,43.26,56.74,0.037951,0.000000,0.032916,0.044307
3,CV-12A,R2011008085,1,0,NaN,0,NaN,52.4100,74.5821,BAT256-k141_32339,...,26.35,30.38,22.57,20.70,43.26,56.74,0.000000,0.048309,0.032916,0.000000
4,CV-12A,R2011008083,8,0,NaN,2,0.232190,443.6190,95.9685,BAT256-k141_32339,...,26.35,30.38,22.57,20.70,43.26,56.74,0.000000,0.096618,0.164582,0.044307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ParV-1A,R2101013077,50,1,0.100070,0,NaN,1829.6200,86.7643,RNALIV2019-k141_21519,...,35.29,25.67,18.26,20.79,39.04,60.96,0.453386,0.865801,0.623296,1.040526
96,ReoV-2B,R2012012467,45,29,0.305499,1,0.076923,74.3356,98.1026,BAT1219-k141_71479,...,29.23,29.59,15.44,25.74,41.18,58.82,0.752651,0.466200,0.405542,0.971503
97,ReoV-2B,R2012012465,5,3,0.064791,2,0.070476,282.4040,100.0000,BAT1219-k141_71479,...,29.23,29.59,15.44,25.74,41.18,58.82,0.034211,0.000000,0.067590,0.064767
98,CV-6A,R2101012916,7,3,0.306084,4,0.316236,171.2540,100.0000,RNAKID1997-k141_187969,...,23.81,20.37,29.16,26.66,55.83,44.17,0.335993,0.037509,0.147275,0.068587


In [68]:
len(isnv_stat_all['vsname'].drop_duplicates())

43

In [ ]:
vardirection_mergebaseperc.drop('sn', axis=1).groupby(['vsname','refid']).mean()#.to_excel('iSNV_vdirection_stat_norm_mergevs.xlsx')

In [103]:
vardirection_mergebaseperc.to_excel('iSNV_vdirection_stat_norm.xlsx', index=None)

In [68]:
pd.DataFrame(statresult, columns=['vsname', 'num_sample', 'num_iSNV']).sort_values(['num_sample', 'num_iSNV'], ascending=False).reset_index(drop=True)

,vsname,num_sample,num_iSNV
0,ParV-1A,17,215
1,CV-2C,6,72
2,CoV-8A,5,191
3,ParV-2A,5,26
4,CoV-8B,5,7
5,CV-14A,4,102
6,CoV-6A,4,9
7,PicoV-4B,3,87
8,CV-12A,3,23
9,CoV-8C,3,2


Compare divergent & non-divergent ns-iSNV allele frequency

In [109]:
isnv_full = pd.read_excel('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/iSNV_full_perPOS_annotated.xlsx')
isnv_stat_filtdp = pd.read_excel('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/iSNV_DPstat_filtCoInf_filtDP_indv.xlsx')
dpdata = isnv_full.merge(isnv_stat_filtdp[['vsname', 'sn']], on=['vsname', 'sn'])
dpdata
dpdata['CHROM_POS'] = dpdata['CHROM'] + '_' + dpdata['POS'].astype(str)
dpdata

,SN,CHROM,POS,REF,ALT,ADPP,Annotation,sn,vsname,CHROM_POS
0,R2101013357,CalV-11A,3405,C,T,0.145695,synonymous_variant,R2101013357,CalV-11A,CalV-11A_3405
1,R2101013357,CalV-11A,6156,C,T,0.101266,synonymous_variant,R2101013357,CalV-11A,CalV-11A_6156
2,R2101013357,CalV-11A,6189,C,T,0.070755,synonymous_variant,R2101013357,CalV-11A,CalV-11A_6189
3,R2101013362,CalV-11A,108,C,T,0.071563,synonymous_variant,R2101013362,CalV-11A,CalV-11A_108
4,R2101013362,CalV-11A,1380,T,C,0.060311,synonymous_variant,R2101013362,CalV-11A,CalV-11A_1380
...,...,...,...,...,...,...,...,...,...,...
1658,R2101012916,CV-6A,1399,A,G,0.427350,missense_variant,R2101012916,CV-6A,CV-6A_1399
1659,R2101012916,CV-6A,1405,A,T,0.356436,missense_variant,R2101012916,CV-6A,CV-6A_1405
1660,R2101012916,CV-6A,1814,T,C,0.340659,synonymous_variant,R2101012916,CV-6A,CV-6A_1814
1661,R2101012909,CV-6A,1387,C,A,0.160987,missense_variant,R2101012909,CV-6A,CV-6A_1387


In [110]:
dpdata = dpdata[dpdata['CHROM'].apply(lambda x: x.startswith('CoV-8A'))]

In [111]:
all_divergent_sites = []
for vsname in ['CoV-8B', 'CoV-8C']:
    msavcf = glob('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/GenomeAnnotation/CoV-8/msa2vcf/CoV-8A_ref{vsprefix}.vcf'.format(vsprefix=vsname.split('-')[-1]))
    divergent_sites = set()
    for vcfpath in msavcf:
        divergent_sites = divergent_sites | set(parsing_sitepos(vcfpath))
    all_divergent_sites += list(map(lambda x: vsname+'_' +str(x), divergent_sites))

In [112]:
all_divergent_sites

['CoV-8B_1',
 'CoV-8B_24577',
 'CoV-8B_24601',
 'CoV-8B_24607',
 'CoV-8B_16425',
 'CoV-8B_24618',
 'CoV-8B_8238',
 'CoV-8B_16431',
 'CoV-8B_8242',
 'CoV-8B_24627',
 'CoV-8B_24630',
 'CoV-8B_24633',
 'CoV-8B_24636',
 'CoV-8B_24639',
 'CoV-8B_24642',
 'CoV-8B_8260',
 'CoV-8B_24645',
 'CoV-8B_24646',
 'CoV-8B_24657',
 'CoV-8B_24660',
 'CoV-8B_24663',
 'CoV-8B_89',
 'CoV-8B_96',
 'CoV-8B_24678',
 'CoV-8B_24681',
 'CoV-8B_24687',
 'CoV-8B_122',
 'CoV-8B_24699',
 'CoV-8B_24700',
 'CoV-8B_128',
 'CoV-8B_24709',
 'CoV-8B_24711',
 'CoV-8B_24714',
 'CoV-8B_16527',
 'CoV-8B_24726',
 'CoV-8B_155',
 'CoV-8B_16539',
 'CoV-8B_24732',
 'CoV-8B_24733',
 'CoV-8B_8356',
 'CoV-8B_8359',
 'CoV-8B_24744',
 'CoV-8B_24747',
 'CoV-8B_24748',
 'CoV-8B_24750',
 'CoV-8B_8368',
 'CoV-8B_24753',
 'CoV-8B_182',
 'CoV-8B_24759',
 'CoV-8B_24765',
 'CoV-8B_190',
 'CoV-8B_24768',
 'CoV-8B_16581',
 'CoV-8B_16582',
 'CoV-8B_16584',
 'CoV-8B_24780',
 'CoV-8B_24781',
 'CoV-8B_8398',
 'CoV-8B_16590',
 'CoV-8B_24783',
 'CoV-8

In [105]:
dpdata_cov8 = dpdata[dpdata['CHROM'].isin(['CoV-8A', 'CoV-8B', 'CoV-8C'])]
dpdata_cov8

,SN,CHROM,POS,REF,ALT,ADPP,Annotation,sn,vsname,CHROM_POS
153,R2101013201,CoV-8B,27941,C,T,0.192825,downstream_gene_variant,R2101013201,CoV-8B,CoV-8B_27941
154,R2101013287,CoV-8B,27941,C,T,0.137566,downstream_gene_variant,R2101013287,CoV-8B,CoV-8B_27941
155,R2101013338,CoV-8B,27941,C,T,0.100000,downstream_gene_variant,R2101013338,CoV-8B,CoV-8B_27941
156,R2101013288,CoV-8B,8056,G,T,0.205128,synonymous_variant,R2101013288,CoV-8B,CoV-8B_8056
157,R2101013288,CoV-8B,21331,G,T,0.052632,synonymous_variant,R2101013288,CoV-8B,CoV-8B_21331
...,...,...,...,...,...,...,...,...,...,...
576,R2012012511,CoV-8A,24160,G,C,0.080645,missense_variant,R2012012511,CoV-8A,CoV-8A_24160
577,R2012012511,CoV-8A,24526,G,T,0.051339,missense_variant,R2012012511,CoV-8A,CoV-8A_24526
578,R2012012506,CoV-8A,21288,G,A,0.050633,synonymous_variant,R2012012506,CoV-8A,CoV-8A_21288
579,R2012012506,CoV-8A,22179,T,C,0.101852,synonymous_variant,R2012012506,CoV-8A,CoV-8A_22179


In [106]:
dpdata_cov8['divergent_pos'] = [  True if pos in all_divergent_sites else False for pos in dpdata_cov8['CHROM_POS'] ]

/tmp/ipykernel_3953/1098738744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dpdata_cov8['divergent_pos'] = [  True if pos in all_divergent_sites else False for pos in dpdata_cov8['CHROM_POS'] ]


In [107]:
dpdata_cov8_nsisnv = dpdata_cov8[dpdata_cov8['Annotation']=='missense_variant']
dpdata_cov8_nsisnv

,SN,CHROM,POS,REF,ALT,ADPP,Annotation,sn,vsname,CHROM_POS,divergent_pos
159,R2101013288,CoV-8B,27413,A,G,0.093264,missense_variant,R2101013288,CoV-8B,CoV-8B_27413,True
160,R2101013286,CoV-8B,20896,G,T,0.147541,missense_variant,R2101013286,CoV-8B,CoV-8B_20896,True
370,R2101013330,CoV-8C,26901,C,T,0.103333,missense_variant,R2101013330,CoV-8C,CoV-8C_26901,True
372,R2101013333,CoV-8C,26901,C,T,0.111801,missense_variant,R2101013333,CoV-8C,CoV-8C_26901,True
519,R2012012513,CoV-8A,6574,G,T,0.384615,missense_variant,R2012012513,CoV-8A,CoV-8A_6574,False
520,R2012012513,CoV-8A,22367,G,T,0.125000,missense_variant,R2012012513,CoV-8A,CoV-8A_22367,False
521,R2012012513,CoV-8A,22417,T,A,0.111650,missense_variant,R2012012513,CoV-8A,CoV-8A_22417,False
522,R2012012513,CoV-8A,23423,G,C,0.068729,missense_variant,R2012012513,CoV-8A,CoV-8A_23423,False
527,R2012012513,CoV-8A,23895,G,T,0.086806,missense_variant,R2012012513,CoV-8A,CoV-8A_23895,False
533,R2012012513,CoV-8A,23915,G,A,0.176471,missense_variant,R2012012513,CoV-8A,CoV-8A_23915,True


In [48]:
dpdata_cov8.to_excel('DP_divergentPOS_inCoV8.xlsx')

,vsname,sn,num_varsite,synonymous_varnum,synonymous_mean,nonsynonymous_varnum,nonsynonymous_mean,meandepth,coverage,SN,CHROM,POS,REF,ALT,ADPP,Annotation,CHROM_POS,divergent_pos
153,CoV-8B,R2101013201,2,0,NaN,0,NaN,246.305,90.0427,R2101013201,CoV-8B,27941,C,T,0.192825,downstream_gene_variant,CoV-8B_27941,True
154,CoV-8B,R2101013287,1,0,NaN,0,NaN,846.018,100.0000,R2101013287,CoV-8B,27941,C,T,0.137566,downstream_gene_variant,CoV-8B_27941,True
155,CoV-8B,R2101013338,1,0,NaN,0,NaN,808.070,99.4059,R2101013338,CoV-8B,27941,C,T,0.100000,downstream_gene_variant,CoV-8B_27941,True
156,CoV-8B,R2101013288,4,3,0.108848,1,0.093264,126.705,99.6229,R2101013288,CoV-8B,8056,G,T,0.205128,synonymous_variant,CoV-8B_8056,False
157,CoV-8B,R2101013288,4,3,0.108848,1,0.093264,126.705,99.6229,R2101013288,CoV-8B,21331,G,T,0.052632,synonymous_variant,CoV-8B_21331,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,CoV-8A,R2012012511,69,6,0.132715,8,0.085771,515.167,98.7956,R2012012511,CoV-8A,24160,G,C,0.080645,missense_variant,CoV-8A_24160,True
586,CoV-8A,R2012012511,69,6,0.132715,8,0.085771,515.167,98.7956,R2012012511,CoV-8A,24526,G,T,0.051339,missense_variant,CoV-8A_24526,True
587,CoV-8A,R2012012506,70,2,0.076242,0,NaN,210.208,97.9997,R2012012506,CoV-8A,21288,G,A,0.050633,synonymous_variant,CoV-8A_21288,True
588,CoV-8A,R2012012506,70,2,0.076242,0,NaN,210.208,97.9997,R2012012506,CoV-8A,22179,T,C,0.101852,synonymous_variant,CoV-8A_22179,True


RAP & VEP region iSNV & cSNV stat

In [4]:
popgenome_refgenome = pd.read_table('requant_group.tsv', names=['sn', 'vsname', 'refseqid'])[['vsname', 'refseqid']].drop_duplicates()
popgenome_refgenome

,vsname,refseqid
0,ParV-7A,QEPC045-k141_1442856
1,FlaV-8A,19BR182KID-k141_466676
3,AstV-2A,UV32-k141_536798
5,PicoV-17B,UGR87-k141_390365
6,CalV-11B,BAT2-k141_981229
...,...,...
792,CalV-3A,BAT272-k141_26345
800,PicoV-7A,BAT0740-k141_8863
802,ParaV-5A,19BR181KID-k99_6171
807,CalV-2A,BAT784-k99_1727


In [6]:
isnvdata = pd.read_excel('/jdfssz1/ST_HEALTH/P20Z10200N0206/renzirui/EA_redo/ViralStrain_PopGenetics/iSNV_DPperPOS_filtered.xlsx')
isnvdata

,vsname,sn,num_varsite,synonymous_varnum,synonymous_mean,nonsynonymous_varnum,nonsynonymous_mean,meandepth,coverage,SN,CHROM,POS,REF,ALT,ADPP,Annotation
0,CalV-11A,R2101013357,3,3,0.105905,0,NaN,322.859,100.0,R2101013357,CalV-11A,3405,C,T,0.145695,synonymous_variant
1,CalV-11A,R2101013357,3,3,0.105905,0,NaN,322.859,100.0,R2101013357,CalV-11A,6156,C,T,0.101266,synonymous_variant
2,CalV-11A,R2101013357,3,3,0.105905,0,NaN,322.859,100.0,R2101013357,CalV-11A,6189,C,T,0.070755,synonymous_variant
3,CalV-11A,R2101013362,7,7,0.187506,0,NaN,628.607,100.0,R2101013362,CalV-11A,108,C,T,0.071563,synonymous_variant
4,CalV-11A,R2101013362,7,7,0.187506,0,NaN,628.607,100.0,R2101013362,CalV-11A,1380,T,C,0.060311,synonymous_variant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,CV-6A,R2101012916,24,3,0.306084,4,0.316236,171.254,100.0,R2101012916,CV-6A,1399,A,G,0.427350,missense_variant
1777,CV-6A,R2101012916,24,3,0.306084,4,0.316236,171.254,100.0,R2101012916,CV-6A,1405,A,T,0.356436,missense_variant
1778,CV-6A,R2101012916,24,3,0.306084,4,0.316236,171.254,100.0,R2101012916,CV-6A,1814,T,C,0.340659,synonymous_variant
1779,CV-6A,R2101012909,2,0,NaN,2,0.108696,1086.840,100.0,R2101012909,CV-6A,1387,C,A,0.160987,missense_variant
